In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import copy
import numpy as np
from models.GreyUTKFace.VAE import Model
import os
from processed_data import GreyUTKFace
import matplotlib.pyplot as plt

In [2]:
%%time
train_dataset = GreyUTKFace.Dataset(train=True)
test_dataset = GreyUTKFace.Dataset(train=False)

CPU times: user 9.57 s, sys: 1.23 s, total: 10.8 s
Wall time: 10.8 s


In [3]:
train_dataset[0][0].shape

torch.Size([1, 128, 128])

In [4]:
model_dir = 'models/GreyUTKFace/VAE/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=128, shuffle=True)

model = Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 1, model.input_w, model.input_h), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    avg_loss = train_loss / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, avg_loss))
    return avg_loss


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(128, 1, model.input_w, model.input_h)[:n]])
                save_image(comparison.cpu(),
                         model_dir + 'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss

def main():
    print(device)
    test_losses = []
    train_losses = []
    weight_path = model_dir + 'weights/model.pt'
    if os.path.exists(weight_path):
        model.load_state_dict(torch.load(weight_path))
        print('Loaded a saved model')
    else:
        print('Starting model from scratch')
    for epoch in range(1, 10 + 1):
        train_loss = train(epoch)
        test_loss = test(epoch)
        
        test_losses.append(test_loss)
        train_losses.append(train_loss)
        
        with torch.no_grad():
            sample = torch.randn(64, model.latent_space).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, model.input_w, model.input_h),
                       model_dir + 'results/sample_' + str(epoch) + '.png')
            
    torch.save(model.state_dict(), weight_path)
    
    plt.plot(np.arange(len(train_losses)), train_losses)
    plt.plot(np.arange(len(test_losses)), test_losses)


In [ ]:
%%time
main()

cuda
Starting model from scratch
Train Epoch: 1 [0/18966 (0%)]	Loss: 12233.429688
